# Experiments for TACAS paper

In [1]:
%%bash
ltl3ba -v
ltl3hoa -v
ltl2tgba --version
ltlcross --version

LTL3BA 1.1.3
LTL3TELA 1.1.0
ltl2tgba (spot 2.4.1.dev)

Copyright (C) 2017  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
ltlcross (spot 2.4.1.dev)

Copyright (C) 2017  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [2]:
from ltlcross_runner import LtlcrossRunner
from experiments_lib import has_f_merging, has_g_merging
from IPython.display import display
import pandas as pd
import spot
import sys
spot.setup(show_default='.a')

In [3]:
def generate(n=100,func=(lambda x: True),filename=None,priorities='M=0,W=0,xor=0',ap=['a','b','c','d','e']):
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(ap,
                     ltl_priorities=priorities,
                     simplify=3,tree_size=15).relabel_bse(spot.Abc)\
                     .unabbreviate('WM')
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

## Impact of F- and G-merging

In [4]:
rerun = False

### Generating Formulae
We generate random formulae from the $LTL(\mathsf{F},\mathsf{G})$ fragment.

In [5]:
fg_f = 'formulae/fg.ltl'
!mkdir -p formulae
generate(500,filename=fg_f,priorities='xor=0,implies=0,equiv=0,X=0,W=0,M=0,R=0,U=0,F=3,G=3')

### Alternating automata

In [6]:
### Tools' setting ###
ltl3hoa_shared = "ltl3hoa -p1 -t0 -u0 -z0 -f %f -a3 -n0 "
end = " > %O"
tools = {"LTL3HOA_both_merging" : ltl3hoa_shared + end,
         "LTL3HOA_F-merging"    : ltl3hoa_shared + "-G0 " + end,
         "LTL3HOA_G-merging"    : ltl3hoa_shared + "-F0" + end,
         "LTL3HOA_no_merging"   : ltl3hoa_shared + "-F0 -G0" + end,
        }
MI_order = ["LTL3HOA_no_merging","LTL3HOA_F-merging",
            "LTL3HOA_G-merging","LTL3HOA_both_merging"]
### File with measured statistics ###
merging_imp_alt = 'MI_alt.csv'

In [7]:
MI_alt = LtlcrossRunner(tools,res_filename=merging_imp_alt,
                        formula_files=[fg_f],
                        cols=["states","transitions"])
if rerun:
    MI_alt.run_ltlcross()
MI_alt.parse_results()

Here are the cumulative number of states and number of nondeterministic automata (without branching transitions) for each tool. 

In [8]:
t1_alt = MI_alt.values.stack(level=0).unstack().sum().unstack().loc[MI_order,:]
t1_alt

column,states,transitions
tool,,
LTL3HOA_no_merging,2467.0,18294.0
LTL3HOA_F-merging,1956.0,14465.0
LTL3HOA_G-merging,1868.0,13932.0
LTL3HOA_both_merging,1362.0,10441.0


### Nondeterministic automata

In [9]:
### Tools' setting ###
ltl3hoa_shared = "ltl3hoa -p2 -s0 -u0 -z0 -f %f -a3 -n0 "
end = " > %O"
tools = {"LTL3HOA_both_merging"       : ltl3hoa_shared + end,
         "LTL3HOA_both_merging_compl" : ltl3hoa_shared + "-n1" + end,
         "LTL3HOA_F-merging"          : ltl3hoa_shared + "-G0" + end,
         "LTL3HOA_G-merging"          : ltl3hoa_shared + "-F0" + end,
         "LTL3HOA_no_merging"         : ltl3hoa_shared + "-F0 -G0" + end,
        }
MI_order = ["LTL3HOA_no_merging","LTL3HOA_F-merging",
            "LTL3HOA_G-merging","LTL3HOA_both_merging",
            "LTL3HOA_both_merging_compl"]
### File with measured statistics ###
merging_imp = 'MI_nondet.csv'
cols=["states","transitions","nondet_aut"]

In [10]:
MI_nondet = LtlcrossRunner(tools,res_filename=merging_imp,
                           formula_files=[fg_f],
                           cols=cols)
if rerun:
    MI_nondet.run_ltlcross()
MI_nondet.parse_results()

Here are the cumulative number of states and number of nondeterministic automata (without branching transitions) for each tool. 

In [21]:
t1_nondet = MI_nondet.values.stack(level=0).unstack().sum().unstack().loc[MI_order,:]
t1_nondet = t1_nondet[cols]; t1_nondet

column,states,transitions,nondet_aut
tool,,,
LTL3HOA_no_merging,1732.0,12289.0,430.0
LTL3HOA_F-merging,1534.0,12313.0,353.0
LTL3HOA_G-merging,1745.0,12378.0,434.0
LTL3HOA_both_merging,1251.0,9588.0,358.0
LTL3HOA_both_merging_compl,1238.0,9522.0,351.0


In [22]:
t1 = pd.concat([t1_alt,t1_nondet],axis=1,keys=['Alternating','Nondeterministic']).loc[MI_order,:]
t1

Alternating             Nondeterministic  \
column                          states transitions           states   
LTL3HOA_no_merging              2467.0     18294.0           1732.0   
LTL3HOA_F-merging               1956.0     14465.0           1534.0   
LTL3HOA_G-merging               1868.0     13932.0           1745.0   
LTL3HOA_both_merging            1362.0     10441.0           1251.0   
LTL3HOA_both_merging_compl         NaN         NaN           1238.0   

                                                   
column                     transitions nondet_aut  
LTL3HOA_no_merging             12289.0      430.0  
LTL3HOA_F-merging              12313.0      353.0  
LTL3HOA_G-merging              12378.0      434.0  
LTL3HOA_both_merging            9588.0      358.0  
LTL3HOA_both_merging_compl      9522.0      351.0

In [23]:
row_map={"LTL3HOA_no_merging" : 'no merging',
         "LTL3HOA_F-merging"  : '$\F$-merging',
         "LTL3HOA_G-merging"  : '$\G$-merging',
         "LTL3HOA_both_merging" : 'full-merging',
         "LTL3HOA_both_merging_compl" : "full-merging + complement"}
t1.rename(row_map,inplace=True); t1

Alternating             Nondeterministic  \
column                         states transitions           states   
no merging                     2467.0     18294.0           1732.0   
$\F$-merging                   1956.0     14465.0           1534.0   
$\G$-merging                   1868.0     13932.0           1745.0   
full-merging                   1362.0     10441.0           1251.0   
full-merging + complement         NaN         NaN           1238.0   

                                                  
column                    transitions nondet_aut  
no merging                    12289.0      430.0  
$\F$-merging                  12313.0      353.0  
$\G$-merging                  12378.0      434.0  
full-merging                   9588.0      358.0  
full-merging + complement      9522.0      351.0

In [24]:
print(t1.to_latex(float_format='{:.0f}'.format,escape=False)
      ,)#file=open('lics_t1.tex','w'))

\begin{tabular}{lrrrrr}
\toprule
{} & \multicolumn{2}{l}{Alternating} & \multicolumn{3}{l}{Nondeterministic} \\
column &      states & transitions &           states & transitions & nondet_aut \\
\midrule
no merging                &        2467 &       18294 &             1732 &       12289 &        430 \\
$\F$-merging              &        1956 &       14465 &             1534 &       12313 &        353 \\
$\G$-merging              &        1868 &       13932 &             1745 &       12378 &        434 \\
full-merging              &        1362 &       10441 &             1251 &        9588 &        358 \\
full-merging + complement &         nan &         nan &             1238 &        9522 &        351 \\
\bottomrule
\end{tabular}



## Comparison with LTL to TGBA translators

### Formulae
We use the same set of 500 formulae from the $LTL(\mathsf{F}\mathsf{G})$ fragment as above.

### Nondeterministic automata

In [25]:
### Tools' setting ###
ltl3hoa_shared = "ltl3hoa -p2 -f %f -a3 -n0 "
end = " > %O"
tools = {"LTL3HOA_both_merging"       : ltl3hoa_shared + end,
         "LTL3HOA_both_merging_compl" : ltl3hoa_shared + "-n1 " + end,
         "LTL3BA"                     : 'ltl3ba -H2 -f %s | autfilt --small > %O',         
         "SPOT-det"                   : 'ltl2tgba --deterministic -H %f>%O',
         "SPOT-small"                 : 'ltl2tgba --small -H %f>%O',
        }
tgba = ["LTL3BA","SPOT-det","SPOT-small"]
### File with measured statistics ###
comp_res = 'comp_nondet.csv'

In [26]:
comp = LtlcrossRunner(tools,res_filename=comp_res,
                           formula_files=[fg_f],
                           cols=cols)
if rerun:
    comp.run_ltlcross()
comp.parse_results()

Here are the cumulative number of states and number of nondeterministic automata (without branching transitions) for each tool. 

In [27]:
comp.compute_best(tgba,'VB(TGBA)')
comp.compute_best(tgba+['LTL3HOA_both_merging'],'VB(no-complement)')
comp.compute_best(None,'VB(All)')

In [28]:
tmp = comp.values.stack(level=0).unstack().sum()
t2 = tmp.unstack().loc[tgba+['LTL3HOA_both_merging',"LTL3HOA_both_merging_compl"]+comp.mins,:][cols]
t2

column,states,transitions,nondet_aut
tool,,,
LTL3BA,1649.0,10767.0,413.0
SPOT-det,1520.0,10051.0,371.0
SPOT-small,1519.0,10048.0,372.0
LTL3HOA_both_merging,1248.0,9544.0,357.0
LTL3HOA_both_merging_compl,1225.0,9410.0,342.0
VB(TGBA),1519.0,9870.0,371.0
VB(no-complement),1194.0,8567.0,309.0
VB(All),1183.0,8504.0,299.0


In [29]:
row_map={"LTL3HOA_both_merging" : 'LTL3HOA-opt',
         "LTL3HOA_both_merging_compl" : "LTL3HOA+opt",
         "VB(no-complement)" : "VB(no-opt)"}
t2.rename(row_map,inplace=True); t2

column,states,transitions,nondet_aut
tool,,,
LTL3BA,1649.0,10767.0,413.0
SPOT-det,1520.0,10051.0,371.0
SPOT-small,1519.0,10048.0,372.0
LTL3HOA-opt,1248.0,9544.0,357.0
LTL3HOA+opt,1225.0,9410.0,342.0
VB(TGBA),1519.0,9870.0,371.0
VB(no-opt),1194.0,8567.0,309.0
VB(All),1183.0,8504.0,299.0


In [20]:
print(t2.to_latex(float_format='{:.0f}'.format,bold_rows=True,
                 ))#,file=open('lics_t2.tex','w'))

\begin{tabular}{lrrr}
\toprule
\textbf{column} &  states &  transitions &  nondet\_aut \\
\textbf{tool       } &         &              &             \\
\midrule
\textbf{LTL3BA     } &    1649 &        10767 &         413 \\
\textbf{SPOT-det   } &    1520 &        10051 &         371 \\
\textbf{SPOT-small } &    1519 &        10048 &         372 \\
\textbf{LTL3HOA-opt} &    1248 &         9544 &         357 \\
\textbf{LTL3HOA+opt} &    1225 &         9410 &         342 \\
\textbf{VB(TGBA)   } &    1519 &         9870 &         371 \\
\textbf{VB(no-opt) } &    1194 &         8567 &         309 \\
\textbf{VB(All)    } &    1183 &         8504 &         299 \\
\bottomrule
\end{tabular}

